In [1]:
import os
os.environ["WANDB_DISABLED"] = "true"

import sys
sys.path.append('..')

%load_ext autoreload
%autoreload 2

In [2]:
import analyze_utils

# inversion model: wandb.ai/jack-morris/emb-inv-1/runs/ebb31d91810c4b62d2b55b5382e8c7ea/logs?workspace=user-jxmorris12
# cross-encoder: wandb.ai/jack-morris/emb-rerank-1/runs/7bd186119e1f4f789eaa0731fd7357a1/overview?workspace=user-jxmorris12
checkpoint_folder = "/home/jxm3/research/retrieval/inversion/saves/98b1418d38c3f9333b17ab20bff06ff9/"

trainer = analyze_utils.load_trainer(checkpoint_folder)

Loading trainer for analysis – setting --do_eval=1
loading alias dpr_nq__msl32_beta from /home/jxm3/research/retrieval/inversion/saves/db66b9c01b644541fedbdcc59c53a285/ebb31d91810c4b62d2b55b5382e8c7ea...
Set train_args.dataloader_num_workers = 4


/home/jxm3/.conda/envs/torch/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


================ Begin trainer sanity check ================
	Input to encode -> Twas brillig, and the slithy toves, Did gyre and gimble in the wabe, All mimsy were the borogoves, And the mome raths outgrabe.
	Decoded output -> The mlbies wase wyst bograge; And the sliths and toms wre
================ End trainer sanity check ================


In [10]:
import torch

model = trainer.inversion_trainer.model
model.use_frozen_embeddings_as_input = True

fake_embedder_input_ids = torch.ones([1], device=trainer.args.device)
fake_embedder_attention_mask = torch.ones([1], device=trainer.args.device)
frozen_embeddings = torch.randn((1, 768), device=trainer.args.device)

tt = model.embedder_tokenizer('This is a random sentence!', return_tensors='pt')
tt = tt.to(trainer.args.device)

output = model.forward(
    embedder_input_ids=fake_embedder_input_ids,
    embedder_attention_mask=fake_embedder_attention_mask,
    frozen_embeddings=frozen_embeddings,
    labels=tt['input_ids']
)

In [ ]:
frozen_embeddings.requires_grad = True

frozen_embeddings = torch.randn((1, 768), device=trainer.args.device, requires_grad=True)
opt = torch.optim.Adam(params=[frozen_embeddings], lr=1)

tt = model.embedder_tokenizer('This is a random sentence!', return_tensors='pt')
tt = tt.to(trainer.args.device)
print('goal text:', model.embedder_tokenizer.decode(tt['input_ids']))

num_steps = 1000
log_interval = 20
for step in range(num_steps):
    output = model.forward(
        embedder_input_ids=fake_embedder_input_ids,
        embedder_attention_mask=fake_embedder_attention_mask,
        frozen_embeddings=frozen_embeddings,
        labels=tt['input_ids']
    )
    gen_tokens = model.generate(
        inputs={
            'embedder_input_ids': fake_embedder_input_ids,
            'embedder_attention_mask': fake_embedder_attention_mask,
            'frozen_embeddings': frozen_embeddings,
        },
        generation_kwargs={
            "max_length": 32,
            "early_stopping": False,
            "num_beams": 1,
            "do_sample": False,
        },
    )
    gen_text = model.embedder_tokenizer.decode(gen_tokens[0])
    if step % log_interval == 0:
        print(f'step {step} loss {output.loss}')
        print(f'\tgen_text = {gen_text}')
    output.loss.backward()
    opt.step()
    
    if loss < 0.5: break

In [ ]:
import tqdm

text = datasets.load_dataset('rotten_tomatoes')['train'][0]['text']
tt = {
    'input_ids': torch.randint(low=1, high=model.embedder_tokenizer.vocab_size, size=(1, 31), dtype=torch.long)
}
tt['input_ids'] = torch.cat((tt['input_ids'], torch.ones((1,1), dtype=torch.long)), dim=1)
tt['attention_mask'] = torch.ones((1, 32))
                            
tt = {k: v.to(trainer.args.device) for k,v in tt.items()}

print('goal text:', model.embedder_tokenizer.decode(tt['input_ids'][0]))

with torch.no_grad():
    frozen_embeddings = model.call_embedding_model(
        **tt
    )
frozen_embeddings.requires_grad = True
# frozen_embeddings = torch.randn((1, 768), device=trainer.args.device, requires_grad=True)

opt = torch.optim.Adam(params=[frozen_embeddings], lr=0.0001)

num_steps = 2000
log_interval = 20

for step in range(0, num_steps):
    output = model.forward(
        embedder_input_ids=fake_embedder_input_ids,
        embedder_attention_mask=fake_embedder_attention_mask,
        frozen_embeddings=frozen_embeddings,
        labels=tt['input_ids']
    )
    gen_tokens = model.generate(
        inputs={
            'embedder_input_ids': fake_embedder_input_ids,
            'embedder_attention_mask': fake_embedder_attention_mask,
            'frozen_embeddings': frozen_embeddings,
        },
        generation_kwargs={
            "max_length": 32,
            "early_stopping": False,
            "num_beams": 1,
            "do_sample": False,
        },
    )
    gen_text = model.embedder_tokenizer.decode(gen_tokens[0])
    if step % log_interval == 0:
        print(f'step {step} loss {output.loss}')
        print(f'\tgen_text = {gen_text}')
    output.loss.backward()
    opt.step()
    
    if output.loss < 0.05: break

  0%|          | 0/3 [00:00<?, ?it/s]

goal text: könnte predominantly rubbish advanced voiciordination Sängerfällig Historical bluntLeaving Restaurant rigid lawmakers Pole simptome convins Monitoring corect 4:00 Nicolerennen DentistryET semnificativ înlocui disciplin ianuarie bleiben nostri wahr</s>
step 0 loss 11.014447212219238
	gen_text = <pad> singing could be asymptomatic Individual vocal flexibility Sängerliche Tätigkeit Bonnie Vorwärts Werbung Werbung Werbung Werbung Werbung Werbung Werbung Werbung Werbung Werbung Vorwärts Regelungen Zusammen
step 20 loss 9.999894142150879
	gen_text = <pad> rigid Can singerături modul modul sonore sonore sonore sonore sonore Werbung Werbung Werbung Werbung Werbung Werbung Zionism Vorwärts et comportament comportament Suceava Suceava Polonia Polonia Polonia
step 40 loss 9.297483444213867
	gen_text = <pad> nearly rigid Bonnie Peut sanatate sanatate sanatate sanatate sanatate sanatate sanatate sanatate sanatate sanatate sanatate sanatate sanatate sanatate sanatate sanatate sanatate san

In [44]:
import datasets
datasets.load_dataset('rotten_tomatoes')['train'][0]['text']

  0%|          | 0/3 [00:00<?, ?it/s]

'the rock is destined to be the 21st century\'s new " conan " and that he\'s going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .'

In [46]:
datasets.load_dataset('rotten_tomatoes')['train'][0]['text']

  0%|          | 0/3 [00:00<?, ?it/s]

'the rock is destined to be the 21st century\'s new " conan " and that he\'s going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .'

In [73]:
# 
# 
# 

# text = datasets.load_dataset('rotten_tomatoes')['train'][0]['text']
# tt = model.embedder_tokenizer(text, return_tensors='pt', max_length=32)
# tt = tt.to(trainer.args.device)
# print('goal text:', model.embedder_tokenizer.decode(tt['input_ids'][0]))

# with torch.no_grad():
#     frozen_embeddings = model.call_embedding_model(
#         **tt
#     )
# frozen_embeddings.requires_grad = True
# # frozen_embeddings = torch.randn((1, 768), device=trainer.args.device, requires_grad=True)

# opt = torch.optim.Adam(params=[frozen_embeddings], lr=0.00001)

num_steps = 2000
log_interval = 20

for step in range(0, num_steps):
    output = model.forward(
        embedder_input_ids=fake_embedder_input_ids,
        embedder_attention_mask=fake_embedder_attention_mask,
        frozen_embeddings=frozen_embeddings,
        labels=tt['input_ids']
    )
    gen_tokens = model.generate(
        inputs={
            'embedder_input_ids': fake_embedder_input_ids,
            'embedder_attention_mask': fake_embedder_attention_mask,
            'frozen_embeddings': frozen_embeddings,
        },
        generation_kwargs={
            "max_length": 32,
            "early_stopping": False,
            "num_beams": 1,
            "do_sample": False,
        },
    )
    gen_text = model.embedder_tokenizer.decode(gen_tokens[0])
    if step % log_interval == 0:
        print(f'step {step} loss {output.loss}')
        print(f'\tgen_text = {gen_text}')
    output.loss.backward()
    opt.step()
    
    if output.loss < 0.05: break

step 0 loss 0.499111145734787
	gen_text = <pad> the rock is destined to be the 21st century's new "" and that he's gonna make a conno
step 20 loss 0.4835469722747803
	gen_text = <pad> the rock is destined to be the 21st century's new "" and that he's gonna make a conno
step 40 loss 0.4662741720676422
	gen_text = <pad> the rock is destined to be the 21st century's new "" and that he's gonna make a conno
step 60 loss 0.4493709206581116
	gen_text = <pad> the rock is destined to be the 21st century's new "" and that he's gonna make a conno
step 80 loss 0.4321572184562683
	gen_text = <pad> the rock is destined to be the 21st century's new "" and that he is gonna make a connoisseur
step 100 loss 0.41421860456466675
	gen_text = <pad> the rock is destined to be the 21st century's new "" and that he is gonna make a connoisseur
step 120 loss 0.39724114537239075
	gen_text = <pad> the rock is destined to be the 21st century's new "" and that he is gonna make a conan and his
step 140 loss 0.3814161

KeyboardInterrupt: 